In [1]:
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm_notebook


In [2]:
# Vivino 탐색을 위한 driver 생성
driver = webdriver.Chrome()
url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NQABteRKW-8gtWQgEa5WAFSQnmZblliUmVqSmKOWX5Rim5JanKyWn1RpW5RYkpmXXhyfWJZalJieqlZeEh1rawgAgG8ccg%3D%3D'
driver.get(url)
time.sleep(3)

In [3]:
# 페이지 끝까지 스크롤하여 모든 데이터 로드
while True:
    # 현재 높이 저장
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 페이지 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 페이지 로드 대기
    time.sleep(1)
    
    # 페이지 살짝 위로 스크롤하여 데이터 로드될 수 있도록 조정함
    driver.execute_script("window.scrollBy(0, -100);")
    time.sleep(3)
    
    #새로운 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height

In [4]:
# 반복문 종료 후 스크롤 맨 위로 올리기
driver.execute_script("window.scrollTo(0,0);")

In [5]:
# 페이지 소스 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 와인 카드 데이터 가져오기 및 개수 확인
wines = soup.find_all('div', class_='wineCard__wineCard--2dj2T')
len(wines)

1961

In [ ]:
# wine 데이터 추출하기

data1 = []
base_url = 'https://www.vivino.com'

for wine in tqdm_notebook(wines):
    
    country = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[-1].strip()
    region = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[0].strip()
    rating = wine.find('div', class_="vivinoRating_averageValue__uDdPM").get_text().strip()
    nratings = wine.find('div', class_="vivinoRating_caption__xL84P").get_text().split()[0].strip() # rating 수 
    label = wine.find('div', class_="wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw").get_text()
    winery = wine.find('div', class_ = "wineInfoVintage__truncate--3QAtw").get_text()
    price = wine.find('div', class_="addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5").get_text().split()[-1].replace('₩', '')

    # Crawling wine varieties
    wine_link = wine.find('a',class_="anchor_anchor__m8Qi- wineCard__cardLink--3F_uB")['href']
    link = base_url + wine_link

    wait = WebDriverWait(driver, 10)
    facts_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'anchor_anchor__m8Qi-')))

    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 맨 밑으로 스크롤하여 모든 페이지소스 로딩
    facts_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'anchor_anchor__m8Qi-'))) # 페이지 로딩될까지 Wait

    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 맨 밑으로 스크롤하여 모든 페이지소스 로딩
    facts_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'anchor_anchor__m8Qi-'))) # 페이지 로딩될까지 Wait
    
    wine_html = driver.page_source
    wine_soup = BeautifulSoup(wine_html, 'html.parser')

    grape_links= wine_soup.find_all('a', class_='anchor_anchor__m8Qi-', href = True)
    grapes = [grape.text for grape in grape_links if '/grapes/' in grape['href']]
    
    # data에 값 저장하기
    data1.append({
        'Country': country,
        'Region': region,
        'Rating': rating,
        'Nratings': nratings,
        'Label': label,
        'Winery':winery,
        'Price': price,
        'Wine_Variety': grapes
    })

df1 = pd.DataFrame(data1)
df1.to_csv('wine_df1.csv', index = False)


In [64]:
driver.quit()

In [81]:
df1.to_csv('vivino_wine_data.csv')